# Example 04: PyTorch Featurization

This notebook demonstrates PyTorch-based atomic structure featurization using Chebyshev descriptors.

**Features:**
- Pure Python/PyTorch implementation (no Fortran)
- GPU acceleration support
- Automatic differentiation for gradients
- Validated against Fortran implementation

In [ ]:
import torch
import numpy as np
from aenet.torch_featurize import ChebyshevDescriptor, BatchedFeaturizer

## 1. Basic Featurization: Water Molecule

Featurize a simple water molecule.

In [ ]:
# Create descriptor
descriptor = ChebyshevDescriptor(
    species=['O', 'H'],
    rad_order=10,      # Radial polynomial order
    rad_cutoff=4.0,    # Radial cutoff (Angstroms)
    ang_order=3,       # Angular polynomial order
    ang_cutoff=1.5     # Angular cutoff (Angstroms)
)

# Water molecule positions
positions = np.array([
    [0.000, 0.000,  0.118],  # O
    [0.000, 0.755, -0.471],  # H
    [0.000, -0.755, -0.471]  # H
])
species = ['O', 'H', 'H']

# Featurize
features = descriptor.featurize_structure(positions, species)

print(f"Feature shape: {features.shape}")
print(f"Number of features per atom: {descriptor.get_n_features()}")
print(f"\nOxygen features (first 10):")
print(features[0, :10])
print(f"\nHydrogen features (first 10):")
print(features[1, :10])

## 2. Understanding Feature Dimensions

Features are organized as: [radial_unwt, angular_unwt, radial_wt, angular_wt]

In [ ]:
# For 2 species with rad_order=10, ang_order=3:
# Radial features: 2 × (10+1) = 22
# Angular features: 2 × (3+1) = 8
# Total: 30 features per atom

rad_unwt = features[:, :11]      # Radial unweighted
ang_unwt = features[:, 11:15]    # Angular unweighted
rad_wt = features[:, 15:26]      # Radial weighted
ang_wt = features[:, 26:30]      # Angular weighted

print("Feature organization:")
print(f"  Radial unweighted:  indices 0-10   (11 features)")
print(f"  Angular unweighted: indices 11-14  (4 features)")
print(f"  Radial weighted:    indices 15-25  (11 features)")
print(f"  Angular weighted:   indices 26-29  (4 features)")

## 3. Periodic System: Crystal Structure

Featurize a crystal with periodic boundary conditions.

In [ ]:
# AuCu crystal structure
positions_pbc = np.array([
    [0.0, 0.0, 0.0],  # Cu
    [0.0, 2.0, 2.0],  # Cu
    [2.0, 0.0, 2.0],  # Au
    [2.0, 2.0, 0.0]   # Au
])
species_pbc = ['Cu', 'Cu', 'Au', 'Au']

# Unit cell
cell = np.array([
    [4.0, 0.0, 0.0],
    [0.0, 4.0, 0.0],
    [0.0, 0.0, 4.0]
])
pbc = np.array([True, True, True])

# Create descriptor for Au-Cu system
descriptor_aucu = ChebyshevDescriptor(
    species=['Au', 'Cu'],
    rad_order=8,
    rad_cutoff=3.5,
    ang_order=5,
    ang_cutoff=3.5
)

# Featurize with PBC
features_pbc = descriptor_aucu.featurize_structure(
    positions_pbc, species_pbc, cell=cell, pbc=pbc
)

print(f"Crystal feature shape: {features_pbc.shape}")
print(f"Cu atom 0 features (first 10): {features_pbc[0, :10]}")
print(f"Au atom 2 features (first 10): {features_pbc[2, :10]}")

## 4. Batch Processing

Efficiently featurize multiple structures.

In [ ]:
# Create batch featurizer
batch_fzer = BatchedFeaturizer(descriptor)

# Multiple water molecules with slight perturbations
batch_positions = [
    torch.tensor(positions, dtype=torch.float64),
    torch.tensor(positions + 0.1 * np.random.randn(*positions.shape),
                 dtype=torch.float64),
    torch.tensor(positions + 0.1 * np.random.randn(*positions.shape),
                 dtype=torch.float64),
]
batch_species = [species, species, species]

# Featurize batch
features_batch, batch_indices = batch_fzer(
    batch_positions, batch_species
)

print(f"Batch features shape: {features_batch.shape}")
print(f"Batch indices shape: {batch_indices.shape}")
print(f"Structure 0 atoms: {(batch_indices == 0).sum().item()}")
print(f"Structure 1 atoms: {(batch_indices == 1).sum().item()}")
print(f"Structure 2 atoms: {(batch_indices == 2).sum().item()}")

## 5. GPU Acceleration

Use GPU for faster featurization (if available).

In [ ]:
if torch.cuda.is_available():
    print("CUDA available - creating GPU descriptor")
    
    # Create descriptor on GPU
    descriptor_gpu = ChebyshevDescriptor(
        species=['O', 'H'],
        rad_order=10,
        rad_cutoff=4.0,
        ang_order=3,
        ang_cutoff=1.5,
        device='cuda'
    )
    
    # Featurize on GPU (input automatically moved)
    features_gpu = descriptor_gpu.featurize_structure(positions, species)
    
    print(f"GPU features shape: {features_gpu.shape}")
    print(f"Features computed on GPU")
else:
    print("CUDA not available - using CPU")

## 6. Gradient Computation

Compute feature gradients for force calculations.

In [ ]:
# Enable gradient tracking
positions_torch = torch.tensor(
    positions, dtype=torch.float64, requires_grad=True
)

# Compute features with gradients
features_torch = descriptor(positions_torch, species)

# Compute gradient via backpropagation
loss = features_torch.sum()
loss.backward()

print("Position gradients:")
print(f"Shape: {positions_torch.grad.shape}")
print(f"Oxygen gradient: {positions_torch.grad[0]}")

## 7. Force Computation from Energy Model

Compute atomic forces from an energy model.

In [ ]:
import torch.nn as nn

# Simple energy model
class EnergyModel(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_features, 64),
            nn.Tanh(),
            nn.Linear(64, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )
    
    def forward(self, features):
        return self.net(features)

# Create and initialize model
model = EnergyModel(descriptor.get_n_features())

# Compute forces: F = -∂E/∂r
positions_for_forces = torch.tensor(positions, dtype=torch.float64)
energy, forces = descriptor.compute_forces_from_energy(
    positions_for_forces, species, model
)

print(f"Total energy: {energy.item():.6f}")
print(f"\nForces on each atom:")
for i, (sp, force) in enumerate(zip(species, forces)):
    print(f"  {sp} {i}: [{force[0]:.6f}, {force[1]:.6f}, {force[2]:.6f}]")

## 8. Integration with Training Pipeline

Example of using features for ML potential training.

In [ ]:
# Simulate training data
n_structures = 10
train_positions = []
train_species = []
train_energies = []

for i in range(n_structures):
    # Add random perturbations to water molecule
    perturbed = positions + 0.2 * np.random.randn(*positions.shape)
    train_positions.append(
        torch.tensor(perturbed, dtype=torch.float64)
    )
    train_species.append(species)
    train_energies.append(torch.randn(1))

# Featurize training set
print("Featurizing training structures...")
train_features = []
for pos, spec in zip(train_positions, train_species):
    feats = descriptor.featurize_structure(
        pos.numpy(), spec
    )
    train_features.append(feats)

print(f"Generated features for {len(train_features)} structures")
print(f"Feature shape per structure: {train_features[0].shape}")

# Features are now ready for training
# model.fit(train_features, train_energies)

## Summary

This notebook demonstrated:
- Basic featurization of isolated molecules
- Periodic system featurization
- Batch processing of multiple structures
- GPU acceleration
- Gradient computation
- Force calculation from energy models
- Integration with training pipelines

The PyTorch implementation provides:
- **No Fortran dependency** - pure Python/PyTorch
- **GPU acceleration** - automatic device handling
- **Automatic differentiation** - for gradient-based methods
- **Validated accuracy** - matches Fortran to machine precision